# imports

https://www.wrl.unsw.edu.au/news/coastsat-how-to-estimate-beach-slopes-using-satellite-imagery
https://www.wrl.unsw.edu.au/research/coastsat


In [1]:
%load_ext autoreload
%autoreload 2
import os
import numpy as np
import pickle
import warnings
import time
warnings.filterwarnings("ignore")
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
from matplotlib import gridspec
plt.ion()
import pandas as pd
from datetime import datetime
from coastsat import SDS_earthcache_api, SDS_preprocess, SDS_shoreline, SDS_tools, SDS_transects, SDS_earthcache_client

## setup timeline and AOL

In [2]:
#setup monthly pipelines
def fmt(x):
    if x < 10:
        return '0' + str(x)
    else:
        return str(x)
def monthly_pipes(startyear,endyear=2024):  
    starts = [] ; ends = []
    dates = range(startyear, endyear)  
    for y in dates:
        for m in range(1, 13):
            starts.append(str(y) + '-' + fmt(m) + '-01')
            if m == 2:
                ends.append(str(y) + '-' + fmt(m) + '-28')
            elif m in [4, 6, 9, 11]:
                ends.append(str(y) + '-' + fmt(m) + '-30')
            else:
                ends.append(str(y) + '-' + fmt(m) + '-31')
    return starts, ends

In [3]:
# these coordinates are a rectangle around the island of Madhirivaadhoo
aoi = { "type": "Polygon", 
        "coordinates": [
      [
        [
          72.94835170895487,
          5.464405141242935
        ],
        [
          72.94835170895487,
          5.452180140477154
        ],
        [
          72.96171014465997,
          5.452180140477154
        ],
        [
          72.96171014465997,
          5.464405141242935
        ],
        [
          72.94835170895487,
          5.464405141242935
        ]
      ]
    ]
    }

# setup pipeline

In [4]:
key = input("please enter your API key for Earthcache:")

In [5]:
repo = 'CoastSat'
root_path = os.getcwd()[ 0 : os.getcwd().find( repo ) + len ( repo )]
cfg_path = os.path.join( root_path, 'earthcache-cfg' )
notebook_client = SDS_earthcache_client.EcClient(cfg_path, key)

d5431cc4-b43b-4f31-8db4-d1efe9e589c9


In [6]:

location_name = 'Madhirivaadhoo_'
# need to specify what type of images want to be downloaded
image_type = "a8fc3dde-a3e8-11e7-9793-ae4260ee3b4b" #ID for true color image
# the available IDs can be found at: https://support.skywatch.com/hc/en-us/articles/7297565063067-Available-Outputs
# look at the SDS_earthcache_api.py file for more options
# like searching, creating a pipeline from a search, and getting the max cost of a pipeline

In [37]:
resolution = [ 'low' ]

In [38]:
# need to initialize the api first
#SDS_earthcache_api.initialize(key)
#create a set of dates so each pipeline is one month
starts,ends = monthly_pipes(2023)
for i in range(0,1):#len(starts)):
    start_date = starts[i]
    print(start_date)
    end_date = ends[i]
    pipeline_name = location_name + start_date
    status, response = notebook_client.calculatePrice(resolution[0], aoi, start_date, end_date)
    if(status != 200):
        print(response['errors'])
    else:
        print(response['data']['archive'])
        print("balance: "+str(response['data']['prepurchase_balance']))
    #add 3 seconds of delay between each request
    time.sleep(3)

2023-01-01
{'interval_quantity': 31, 'price_km2': 0.0, 'minimum_area_km2': 0.0, 'interval_cost': 0.0, 'max_cost': 0.0}
balance: 3000.0


In [36]:
key = input("please enter your API key for Earthcache:")
interval = 0
start_date = starts[interval]
end_date = ends[interval]
pipeline_name = location_name + start_date
SDS_earthcache_api.initialize(key)
#SDS_earthcache_api.retrieve_images_earthcache(key, pipeline_name, aoi, start_date, end_date, image_type)

d5431cc4-b43b-4f31-8db4-d1efe9e589c9
201 {'data': {'id': '1aaebb2e-eae4-11ee-9b07-2333090573a3', 'name': 'Madhirivaadhoo_2023-01-01', 'start_date': '2023-01-01', 'end_date': '2023-01-31', 'aoi': {'type': 'Polygon', 'coordinates': [[[72.94835170895487, 5.464405141242935], [72.94835170895487, 5.452180140477154], [72.96171014465997, 5.452180140477154], [72.96171014465997, 5.464405141242935], [72.94835170895487, 5.464405141242935]]]}, 'area_km2': 2.0145276052499086, 'cloud_cover_percentage': 100, 'min_aoi_coverage_percentage': 0, 'interval': '30d', 'resolution_low': 30, 'resolution_high': 10, 'status': 'pending', 'tags': None, 'created_at': '2024-03-25T20:13:11.534188Z', 'updated_at': '2024-03-25T20:13:11.534188Z', 'max_cost': 0, 'result_delivery': {'max_latency': '0d', 'priorities': ['lowest_cost']}, 'output': {'id': 'a8fc3dde-a3e8-11e7-9793-ae4260ee3b4b', 'format': 'geotiff', 'mosaic': 'stitched'}}}
200 {'data': {'id': '1aaebb2e-eae4-11ee-9b07-2333090573a3', 'org_id': '001JR00000AfBY1YAN

In [39]:
# after about 20 minutes, make sure the status is ready
SDS_earthcache_api.initialize(key)
for i in range(len(starts)):
    start_date = starts[i]
    end_date = ends[i]
    pipeline_name = location_name + start_date
    status,res = SDS_earthcache_api.checkStatus(pipeline_name)
    print(status,res)

d5431cc4-b43b-4f31-8db4-d1efe9e589c9
200 {'data': [{'id': 'f8b03317-15c7-4306-98cc-bdb835dea5ee', 'created_at': '2024-03-25T20:13:30.956731+0000', 'updated_at': '2024-03-25T20:14:17.577790+0000', 'pipeline_id': '1aaebb2e-eae4-11ee-9b07-2333090573a3', 'output_id': 'a8fc3dde-a3e8-11e7-9793-ae4260ee3b4b', 'status': 'processing', 'message': 'Image processing - started', 'interval': {'start_date': '2023-01-01', 'end_date': '2023-01-30'}, 'total_interval_cost': None, 'overall_metadata': {'scene_height': None, 'scene_width': None, 'filled_area_km2': None, 'filled_area_percentage_of_aoi': None, 'cloud_cover_percentage': None, 'cloud_cover_percentage_of_aoi': None, 'visible_area_km2': None, 'visible_area_percentage': None, 'visible_area_percentage_of_aoi': None}, 'results': [], 'alternate_search_results': []}, {'id': 'db58e59b-e0ea-46eb-870c-57eaf957aea1', 'created_at': '2024-03-25T20:13:30.914977+0000', 'updated_at': '2024-03-25T20:13:51.902373+0000', 'pipeline_id': '1aaebb2e-eae4-11ee-9b07-23

In [ ]:
# once the status is good, run this cell to download the images
SDS_earthcache_api.initialize(key)
images = []
for i in range(len(starts)):
    start_date = starts[i]
    pipeline_name = location_name + start_date
    status,res = SDS_earthcache_api.checkStatus(pipeline_name)
    if(status == 200):
        images.append(SDS_earthcache_api.download_images(pipeline_name))

In [ ]:
# this function visualizes the first image
SDS_earthcache_api.view_first_image(images)